# Pipeline Overview

1. Data loader (Preprocessed)
2. Document embedding
- TF/IDF
3. Dimension Reduction
- PCA
4. Clustering
- K-means
5. Topic extraction
- LDA

https://g.co/bard/share/b2aabc2237dd

In [1]:
!python --version


Python 3.10.12


# Import Library

In [2]:
!pip install pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 88.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 38.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cupy-cuda11x 11.0.0 requires numpy<1.26,>=1.20, but you have numpy 1.26.1 which is incompatible.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.1.1 which is incompatible.
numba 0.5

In [3]:
!pip install numpy==1.23.5
!pip install pandas==1.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 60.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.1
    Uninstalling numpy-1.26.1:
      Successfully uninstalled numpy-1.26.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyldavis 3.4.1 requires numpy>=1.24.2, but you have numpy 1.23.5 which is incompatible.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 42.9 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.1.1
    Uninstalling pandas-2.1.1:
      Successfully uninstalled pandas-2.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyldavis 3.4.1 requires numpy>=1.24.2, but you have numpy 1.23.5 which is incompatible.
pyldavis 3.4.1 requires pandas>=2.0.0, but you have pandas 1.5.3 which is incompatible.


In [4]:
!pip install hdbscan
!pip install python-box
!pip install joblib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 14.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Cython-0.29.36-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
  Created wheel for hdbscan: filename=hdbscan-0.8.33-cp310-cp310-linux_x86_64.whl size=3039165 sha256=2a0d397840e731fc6f56b4751e6589b3b3eaec1520d75b193a6998da4c9ec13a
  Stored in directory: /root/.cache/pip/wheels/75/0b/3b/dc4f60b7cc455efaefb62883a7483e76f09d06ca81cf87d610
Successfully built hdbscan
  Attempting uninstall: cython
    Found existing installation: Cython 3.0.3
    Uninstalling Cython-3.0.3:
      Successfully uninstalled Cython-3.0.3


In [5]:
import numpy as np
import pandas as pd

# Sklearn
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE
from sklearn.decomposition import LatentDirichletAllocation
from scipy import stats
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
from sklearn import mixture
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import silhouette_samples, silhouette_score, make_scorer
from sklearn.model_selection import RandomizedSearchCV

# Clustering
import hdbscan
from scipy.spatial import distance # To calculate distances
import scipy.cluster.hierarchy as sch
from joblib import Memory

# Visualization
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns

# Topic modeling
import gensim
from gensim import corpora
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

from box import Box


In [6]:
config = {
    "random_state": 42,
    "document_embedding": {
        "max_features": 6000 # 데이터셋 크기 축소에 따라 수정 10000 # 환경에 따라 메모리 부족할 수도 있음. 터지면 줄여서 사용 요망
    },
    "dim_reduction": {
        "n_components": 180, #50,
    },
    "clustering": {
        "n_clusters": 3
    },
    "lda": {
        "n_components": 10,
        "num_keywords": 30,
    }

    # ctfidf 부분 추가 필요
}

config = Box(config)
config # config.pca.n_components의 형태로 접근 가능


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Box({'random_state': 42, 'document_embedding': {'max_features': 6000}, 'dim_reduction': {'n_components': 180}, 'clustering': {'n_clusters': 3}, 'lda': {'n_components': 10, 'num_keywords': 30}})

# Load data

이 공유 드라이브에 대해 "정리->바로가기 추가"를 통해

내 드라이브 root에 공유 드라이브 바로가기를 만드셔야해요

제출할 때는 압축해서 낼 거니까 이 부분은 없애고 ./ 로 바꿔서 낼게요



In [7]:
# mount Google drive
from google.colab import drive
drive.mount('/content/drive')

# now you can see files
!echo -e "\nNumber of Google drive files in /content/drive/My Drive/:"
!ls -a "/content/drive/MyDrive/23-2_기계학습_팀플"
# by the way, you can run any linux command by putting a ! at the start of the line

# by default everything gets executed and saved in /content/
!echo -e "\nCurrent directory:"
!pwd


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/pexpect/popen_spawn.py:60: DeprecationWarning: setDaemon() is deprecated, set the daemon attribute instead
  self._read_thread.setDaemon(True)


Mounted at /content/drive

Number of Google drive files in /content/drive/My Drive/:
'9조 Mid_제출ver.docx'	 dataset	   Final.ipynb		    '기계학습 Mid-Report.gdoc'
'9조 Mid_제출ver.pdf'	 EDA_count.ipynb   mid_report.ipynb	    '기계학습 Proposal.gdoc'
 BioBERT.ipynb		 EDA.ipynb	   Pipeline.ipynb
 cache			 experiments	  'Related Work List.gdoc'

Current directory:
/content


### Set your workspace path

In [8]:
workspace_path = '/content/drive/MyDrive/23-2_기계학습_팀플'  # Change this path!
# workspace_path = './';
filename = 'full_beauty_w_raw_rs.csv'
print(f'Current Workspace: {workspace_path}')

try:
  import google.colab
  data_path = f'{workspace_path}/dataset/{filename}'
  cachedir = f'{workspace_path}/cache'
except:
  data_path = f'./dataset/{filename}'



Current Workspace: /content/drive/MyDrive/23-2_기계학습_팀플


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
df = pd.read_csv(data_path)
df.head()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Unnamed: 0,review,category,rawReview
0,60616,disregard claim hear commercial reduce stretch...,beauty,Disregard all the claims you've heard on comme...
1,72986,price pretty good find nozzle attachment fall ...,beauty,the price it's pretty good. I find the nozzle ...
2,15877,look like harden vasoline come tube smell like...,beauty,It looks like hardened Vasoline coming from th...
3,85030,absolutely love thing small face short eyebrow...,beauty,I absolutely love this the only thing is that ...
4,83073,love set brush two favorite brush set blush b...,beauty,I love this set of brushes! My two favorite br...


# 결측치 제거

In [10]:
df[df["review"].isnull()]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Unnamed: 0,review,category,rawReview


In [11]:
print(df["review"].isnull().sum())
df = df[df["review"].notnull()].reset_index(drop=True)
df.head()


0


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Unnamed: 0,review,category,rawReview
0,60616,disregard claim hear commercial reduce stretch...,beauty,Disregard all the claims you've heard on comme...
1,72986,price pretty good find nozzle attachment fall ...,beauty,the price it's pretty good. I find the nozzle ...
2,15877,look like harden vasoline come tube smell like...,beauty,It looks like hardened Vasoline coming from th...
3,85030,absolutely love thing small face short eyebrow...,beauty,I absolutely love this the only thing is that ...
4,83073,love set brush two favorite brush set blush b...,beauty,I love this set of brushes! My two favorite br...


In [12]:
category2id = {v:id for id, v in enumerate(df['category'].unique())}
id2category = {id:v for id, v in enumerate(df['category'].unique())}


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
df["category_id"] = df["category"].map(category2id)
df.head()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Unnamed: 0,review,category,rawReview,category_id
0,60616,disregard claim hear commercial reduce stretch...,beauty,Disregard all the claims you've heard on comme...,0
1,72986,price pretty good find nozzle attachment fall ...,beauty,the price it's pretty good. I find the nozzle ...,0
2,15877,look like harden vasoline come tube smell like...,beauty,It looks like hardened Vasoline coming from th...,0
3,85030,absolutely love thing small face short eyebrow...,beauty,I absolutely love this the only thing is that ...,0
4,83073,love set brush two favorite brush set blush b...,beauty,I love this set of brushes! My two favorite br...,0


# Document embedding

### TF-IDF

In [ ]:
corpus = df["review"]
embedding_method = TfidfVectorizer(max_features=config.document_embedding.max_features)
tfidf_vec = embedding_method.fit_transform(corpus)
df["tfidf_vec"] = list(tfidf_vec.toarray())


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
df.head()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Unnamed: 0,review,category,rawReview,category_id,tfidf_vec
0,60616,disregard claim hear commercial reduce stretch...,beauty,Disregard all the claims you've heard on comme...,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,72986,price pretty good find nozzle attachment fall ...,beauty,the price it's pretty good. I find the nozzle ...,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,15877,look like harden vasoline come tube smell like...,beauty,It looks like hardened Vasoline coming from th...,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,85030,absolutely love thing small face short eyebrow...,beauty,I absolutely love this the only thing is that ...,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,83073,love set brush two favorite brush set blush b...,beauty,I love this set of brushes! My two favorite br...,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


## SentenceTransformer

In [14]:
!pip install -U sentence-transformers
!pip install -e .

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 56.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.9 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=542ff13930a4f98010f77acb78684bb3197d8465547519d26b5d8f646d806e90
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
Obtaining file:///content
ERROR: file:///content does not appear to 

In [16]:
from sentence_transformers.readers import InputExample
from sentence_transformers import SentenceTransformer, models
from tqdm import tqdm
# download pretrained model
sentence_model = SentenceTransformer('all-MiniLM-L6-v2')

#df["sbert"] = df.apply(lambda row: model.encode(row.review), axis = 1)
sbert = sentence_model.encode(df["review"])
df["sbert"] = list(sbert) # list(sbert.toarray())
df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Unnamed: 0,review,category,rawReview,category_id,sbert
0,60616,disregard claim hear commercial reduce stretch...,beauty,Disregard all the claims you've heard on comme...,0,"[-0.0021139544, 0.031628005, 0.053724263, 0.05..."
1,72986,price pretty good find nozzle attachment fall ...,beauty,the price it's pretty good. I find the nozzle ...,0,"[-0.084313385, -0.02356731, 0.07364636, 0.0112..."
2,15877,look like harden vasoline come tube smell like...,beauty,It looks like hardened Vasoline coming from th...,0,"[-0.046834804, -0.041300334, 0.03862773, 0.041..."
3,85030,absolutely love thing small face short eyebrow...,beauty,I absolutely love this the only thing is that ...,0,"[-0.009914572, 0.012472473, 0.012321725, -0.00..."
4,83073,love set brush two favorite brush set blush b...,beauty,I love this set of brushes! My two favorite br...,0,"[-0.12224284, -0.067056805, 0.024126083, -0.01..."


# Dimension Reduction

#### TruncatedSVD

In [17]:
# dim_reduction_model = PCA(n_components=config.pca.n_components, random_state=config.random_state)
dim_reduction_model = TruncatedSVD(n_components=config.dim_reduction.n_components, random_state=config.random_state)
#dim_reduced = dim_reduction_model.fit_transform(tfidf_vec)
dim_reduced = dim_reduction_model.fit_transform(sbert) # sbert 시 dim_reduction.n_components >=180이면 PoV >=0.9

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
dim_reduced.shape

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(30000, 180)

In [19]:
proportion_of_variance = np.sum(dim_reduction_model.explained_variance_ratio_)
proportion_of_variance


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.919511

#### UMAP

In [ ]:
!pip uninstall umap
!pip install umap-learn
!pip install umap
!pip install -e
!pip install datashader bokeh holoviews scikit-image and colorcet

In [ ]:
import umap.umap_ as umap
import umap.plot
#import umap.plot
mapper = umap.UMAP(densmap=True, n_components=2, random_state=42)
umap_emb = mapper.fit_transform(dim_reduced)

mapped = mapper.fit(umap_emb.data)
umap.plot.points(mapper)

## Visualization (with Category labels)

In [ ]:
# tnse = TSNE(n_components=2, random_state=config.random_state) # 시간 매우 오래 걸림. 주의
# tnse_reduced = tnse.fit_transform(dim_reduced)


In [ ]:
# df["tnse_reduced_0"] = tnse_reduced[:, 0]
# df["tnse_reduced_1"] = tnse_reduced[:, 1]


In [ ]:
# category_df = df[df["category_id"] == 2]


In [ ]:
# sns.scatterplot(x=tnse_reduced[:, 0], y=tnse_reduced[:, 1], hue=df["category_id"], palette="Set2")


# Clustering

클러스터링 알고리즘의 종류: https://scikit-learn.org/stable/modules/clustering.html#overview-of-clustering-methods

### K Means

In [ ]:
clustering_model = KMeans(n_clusters=10)



In [ ]:
largest_cluster_counts = []

for i in range(0, 100):
	clustered = clustering_model.fit_predict(dim_reduced)
	largest_cluster_counts.append(pd.Series(clustered).value_counts().iloc[0])

print(np.var(largest_cluster_counts))
del largest_cluster_counts

random_state 를 넣어서 non determinisitic하게 만들 수 있으나, 이는 주어지는 데이터에 대한 성능 분석으로 조절할 수 있는 hyperparameter가 아님.
우리의 task는 문서를 수학적 기준으로 일관되게 분류하는 것이기 때문에, deterministic한 알고리즘을 사용해야한다고 판단.

### Metric: Silhouette score

### Agglomerative (bottom-up)

Agglomerative Clustering 에 대한 설명: https://process-mining.tistory.com/123
- `Code Reference`: https://colab.research.google.com/github/sandipanpaul21/ML-Clustering-in-Python/blob/master/09_Hierarchical_Clustering.ipynb#scrollTo=qcrOZvxZDrFS

In [ ]:
# Agglomerative Clustering

# 본 알고리즘 또 클러스터의 개수를 하이퍼파라미터로 지정해주어야 한다.
# clustering_model = DBSCAN(eps=1,min_samples=6)
# clustered = clustering_model.fit_predict(dim_reduced)

# Building an Agglomerative Clustering Model
# decide that we will be making 3 clusters for this dataset (클러스터의 개수 임의 설정)
cluster_H = AgglomerativeClustering(n_clusters=3,linkage = 'average')

# Fitting Model
model_clt = cluster_H.fit(dim_reduced)
print(model_clt)
print('\n')

temp = dim_reduced

temp['clusters'] = model_clt.labels_
print('Clusters assigned to each datapoints, cluster = 3 :')
print(temp['clusters'])


1. (T4 기준) 고용량 RAM 을 사용해도 계속 세션이 다운된다. 특히 linkage 단계에서 계속 세션이 다운되는데, linkage 의 3가지 종류 중, RAM 사용량이 비교적 적은 방법론을 찾아봐야 할 것 같다. - 관련없다고 한다.

The choice of linkage method in agglomerative clustering (e.g., single, complete, average, etc.) typically does not have a direct impact on RAM usage. The linkage method primarily affects how clusters are merged during the agglomeration process and the resulting cluster structure.

RAM usage is more closely related to the size of your dataset and the specific clustering algorithm's memory requirements.


2. n_features 및 n_components 의 개수 최소화한 상태에서, n_cluster 의 개수를 조정한다. \
(일단 클러스터링을 한 번이라도 제대로 돌려보는 것이 목표이다.)
*  N = 10 :  RAM 다운
*  N = 5 : RAM 다운
*  N = 3 : RAM 다운, 약 10분간 돌렸는데 RAM 80기가 소요
\

> RAM 이슈를 해결하기 위해 구글링을 해결해본 결과, 본 알고리즘은 알고리즘 자체의 시간 복잡도가 너무 높아서 large dataset 에는 부적합한 알고리즘이라고 한다. (실제로 코랩 프로플러스의 83.5 GB RAM 에서도 자꾸 터진다.) \
참고: https://datascience.stackexchange.com/questions/47889/how-to-run-agglomerativeclustering-on-a-big-data-in-python



3. 우리의 데이터셋에 맞는 number of clusters 를 결정하기 위해 dendrogram 을 활용한다. \
(dendrogram 이란 arrangement of clusters 를 보여주는 트리이다.) \
그래프를 보며, 큰 클러스터링 묶음에 대한 감을 잡을 수 있다. \
 다시 말해, 트리가 너무 작은 단위로 쪼개지기 직전 단계에서 트리를 분할해주는 클러스터의 개수를 찾는다.

In [ ]:
import scipy.cluster.hierarchy as sch


In [ ]:
# Plotting of Dendrogram (average 버전)

Z = sch.linkage(dim_reduced, method = 'average') # Avg Distance between all points
plt.figure(figsize=(20,7))
den = sch.dendrogram(Z)
plt.title('Dendrogram for the agg.. clustering of the dataset')
plt.xlabel('Category')
plt.ylabel('Euclidean distance in the space with other variables')


In [ ]:
# Plotting of Dendrogram (single 버전)

Z = sch.linkage(dim_reduced, method = 'single') # Nearest Point
plt.figure(figsize=(20,7))
den = sch.dendrogram(Z)
plt.title('Dendrogram for the agg.. clustering of the dataset')
plt.xlabel('Category')
plt.ylabel('Euclidean distance in the space with other variables')


In [ ]:
# Plotting of Dendrogram (complete 버전)

Z = sch.linkage(dim_reduced, method = 'complete') # Farthest Point
plt.figure(figsize=(20,7))
den = sch.dendrogram(Z)
plt.title('Dendrogram for the agg.. clustering of the dataset')
plt.xlabel('Category')
plt.ylabel('Euclidean distance in the space with other variables')


### Gaussian Mixture

`Reference`: https://colab.research.google.com/github/sandipanpaul21/ML-Clustering-in-Python/blob/master/13_Gaussian_Mixture_Model.ipynb

In [ ]:
gmm = mixture.GaussianMixture(n_components=3).fit(dim_reduced) # n_components: 3
labels = gmm.predict(dim_reduced)
plt.scatter(dim_reduced[:, 0], dim_reduced[:, 1], c=labels, s=40, cmap='viridis');

# 실루엣 스코어
silhouette_avg = silhouette_score(dim_reduced, labels)
print("Silhouette Score for GMM 3:", silhouette_avg)

In [ ]:
gmm = mixture.GaussianMixture(n_components=4).fit(dim_reduced) # n_components: 4
labels = gmm.predict(dim_reduced)
plt.scatter(dim_reduced[:, 0], dim_reduced[:, 1], c=labels, s=40, cmap='viridis');

# 실루엣 스코어
silhouette_avg = silhouette_score(dim_reduced, labels)
print("Silhouette Score for GMM 4:", silhouette_avg)

In [ ]:
gmm = mixture.GaussianMixture(n_components=5).fit(dim_reduced) # n_components: 5
labels = gmm.predict(dim_reduced)
plt.scatter(dim_reduced[:, 0], dim_reduced[:, 1], c=labels, s=40, cmap='viridis');

# 실루엣 스코어
silhouette_avg = silhouette_score(dim_reduced, labels)
print("Silhouette Score for GMM 5:", silhouette_avg)

In [ ]:
gmm = mixture.GaussianMixture(n_components=10).fit(dim_reduced) # n_components: 클러스트의 개수
labels = gmm.predict(dim_reduced)
plt.scatter(dim_reduced[:, 0], dim_reduced[:, 1], c=labels, s=40, cmap='viridis');

1. 클러스터의 개수를 잘 조정할 필요가 있다.
> How many components?
The fact that GMM is a generative model gives us a natural means of determining the optimal number of components for a given dataset.
A generative model is inherently a probability distribution for the dataset, and so we can simply evaluate the likelihood of the data under the model, using cross-validation to avoid over-fitting.
Another means of correcting for over-fitting is to adjust the model likelihoods using some analytic criterion such as the Akaike information criterion (AIC) or the Bayesian information criterion (BIC)
Scikit-Learn's GMM estimator actually includes built-in methods that compute both of these.

\
The optimal number of clusters is the value that minimizes the AIC or BIC, depending on which approximation we wish to use.


2. 클러스터의 개수가 증가할수록 실루엣 계수 값이 줄어든다. (BAD)

In [ ]:
# 최적의 n_components 찾기 w/ 사이킷런 라이브러리

# n_components_gmm = np.arange(1, 21)
n_components_gmm = np.arange(1, 51)
models = [mixture.GaussianMixture(n, covariance_type='full', random_state=0).fit(dim_reduced)
          for n in n_components_gmm]

plt.plot(n_components_gmm, [m.bic(dim_reduced) for m in models], label='BIC')
plt.plot(n_components_gmm, [m.aic(dim_reduced) for m in models], label='AIC')
plt.legend(loc='best')
plt.xlabel('n_components');

In [ ]:
# 최적의 n_components 찾기 w/ 사이킷런 라이브러리

# n_components_gmm = np.arange(1, 21)
# n_components_gmm = np.arange(1, 51)
n_components_gmm = np.arange(1, 101)

models = [mixture.GaussianMixture(n, covariance_type='full', random_state=0).fit(dim_reduced)
          for n in n_components_gmm]

plt.plot(n_components_gmm, [m.bic(dim_reduced) for m in models], label='BIC')
plt.plot(n_components_gmm, [m.aic(dim_reduced) for m in models], label='AIC')
plt.legend(loc='best')
plt.xlabel('n_components');

타 클러스터링 알고리즘에 비해 (PLOT 상으로는) 성능이 우수한지 잘 모르겠다. 한편, GMM 알고리즘에서 optimal 한 n_components 수를 찾기 위해, BIC 및 AIC 의 변화를 보았으나 AIC는 거의 차이가 없고 (elbow 가 발생하지 않고) BIC 기준으로는 n_components 가 약 3-4개일 때가 최적으로 유추된다.

### DBSCAN

In [ ]:
# clustering_model = DBSCAN(eps=1,min_samples=6)
# clustered = clustering_model.fit_predict(dim_reduced)


대표적으로 K 값을 데이터 분포에 맞게 만들어주는 DBSCAN을 사용해보았으나, DBSCAN은 메모리 복잡도가 너무 커서 100,000만개의 데이터에 적용하는데 적합하지 않았다. (커널 메모리 최대치를 너무 쉽게 넘는다)

따라서 비슷한 작업을 하지만 알고리즘이 달라 메모리 복잡도가 더 낮은 HDBSCAN을 사용하기로 하였다.

### HDBSCAN

Hyperparameter tuning

https://towardsdatascience.com/tuning-with-hdbscan-149865ac2970

In [20]:
memory = Memory(cachedir, verbose=0)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [21]:
import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
clusterer = hdbscan.HDBSCAN(gen_min_span_tree=True, memory=memory).fit(dim_reduced)

# specify parameters and distributions to sample from
param_dist = {'min_samples': [10,30,50,60,100],
              'min_cluster_size':[100,200,300,400,500],
             }

#validity_scroer = "hdbscan__hdbscan___HDBSCAN__validity_index"
validity_scorer = make_scorer(hdbscan.validity.validity_index,greater_is_better=True)


n_iter_search = 20
random_search = RandomizedSearchCV(clusterer
                                   ,param_distributions=param_dist
                                   ,n_iter=n_iter_search
                                   ,scoring=validity_scorer
                                   ,random_state=42)

random_search.fit(dim_reduced)

In [ ]:
print(f"Best Parameters {random_search.best_params_}")
print(f"DBCV score :{random_search.best_estimator_.relative_validity_}")

# evalute the clusters
labels = random_search.best_estimator_.labels_
print(labels)
clustered = (labels >= 0)

coverage = np.sum(clustered) / dim_reduced.shape[0]
total_clusters = np.max(labels) + 1
cluster_sizes = np.bincount(labels[clustered]).tolist()

print(f"Percent of data retained: {coverage}")
print(f"Total Clusters found: {total_clusters}")
print(f"Cluster splits: {cluster_sizes}")

Best Parameters {'min_samples': 60, 'min_cluster_size': 200}
DBCV score :HDBSCAN(gen_min_span_tree=True,
        memory=Memory(location=/content/drive/MyDrive/23-2_기계학습_팀플/cache/joblib),
        min_cluster_size=200, min_samples=60)
[-1 -1 -1 ... -1 -1 -1]
Percent of data retained: 0.0
Total Clusters found: 0
Cluster splits: []


In [ ]:
df["clustered"] = labels
df["clustered"].value_counts()

In [ ]:
corpus = df["review"] # corpus 가 정의되어 있지 않길래, 추가했습니다. - 윤진
d={"review": corpus, "clustered": pd.Series(clustered)}
cluster_result = pd.DataFrame(data=d)

for i in range(len(pd.Series(clustered).unique())):
  print(f'{i}th cluster size: {len(cluster_result[cluster_result["clustered"] == i])}')

## Visualization (with clusters)

In [ ]:
# 2차원 축소 필요
num_category = len(df["category_id"].unique())
fig, axes = plt.subplots(1, num_category, figsize=(15, 5))

for category_id in range(num_category):
    category_df = df[df["category_id"] == category_id]
    axes[category_id].set_title(f"{id2category[category_id]}")
    sns.scatterplot(x=category_df["tnse_reduced_0"], y=category_df["tnse_reduced_1"], hue=category_df["cluster_num"], palette="Set2", ax=axes[category_id])


In [ ]:
# plot two sns scatter plots side-by-side
fig, axes = plt.subplots(1, 2, figsize=(15, 5))
sns.scatterplot(x=tnse_reduced[:, 0], y=tnse_reduced[:, 1], hue=df["category_id"], palette="Set2", ax=axes[0])
sns.scatterplot(x=tnse_reduced[:, 0], y=tnse_reduced[:, 1], hue=df["cluster_num"], palette="Set2", ax=axes[1])


# Topic Modeling (Using Gensim)

In [ ]:
df["review_tokenized"] = df["review"].apply(lambda x: x.split())
df.head()

### Hyperparameter Tuning

In [ ]:
# Find best 'passes' parameter by topic coherence score
coherence_score = []
num_passes_list = np.arange(1, 31, 5)

tokenized_text = df["review_tokenized"]
dictionary = corpora.Dictionary(tokenized_text)
corpus = [dictionary.doc2bow(text) for text in tokenized_text]
num_topics = 10
for num_passes in num_passes_list:
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                                id2word=dictionary,
                                                num_topics=num_topics,
                                                passes=num_passes,
                                                random_state=config.random_state)

    # Compute Coherence Score
    coherence_model_lda = gensim.models.CoherenceModel(model=lda_model, texts=tokenized_text.tolist(), coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print(f"num_topics={num_topics}, passes={num_passes}, Coherence Score={coherence_lda:.4f}")
    coherence_score.append(coherence_lda)
    del lda_model

In [ ]:
# plotting coherence score
plt.plot(num_passes_list, coherence_score)
plt.xlabel("Num Passes")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
# Find best 'num_topics' parameter by topic coherence score
coherence_score = []
num_topics_list = np.arange(5, 31, 5)

tokenized_text = df["review_tokenized"]
dictionary = corpora.Dictionary(tokenized_text)
corpus = [dictionary.doc2bow(text) for text in tokenized_text]
passes = 10
for num_topics in num_topics_list:
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                                id2word=dictionary,
                                                num_topics=num_topics,
                                                passes=passes,
                                                random_state=config.random_state)

    # Compute Coherence Score
    coherence_model_lda = gensim.models.CoherenceModel(model=lda_model, texts=tokenized_text.tolist(), coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print(f"num_topics={num_topics}, passes={num_passes}, Coherence Score={coherence_lda:.4f}")
    coherence_score.append(coherence_lda)
    del lda_model

In [ ]:
# plotting coherence score
plt.plot(num_topics_list, coherence_score)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

### Extract Keywords

In [ ]:
corpus = df["review_tokenized"]
dictionary = corpora.Dictionary(corpus)
corpus = [dictionary.doc2bow(text) for text in corpus]
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=dictionary,
                                            num_topics=15,
                                            passes=10,
                                            random_state=config.random_state)


In [ ]:
def extract_keywords_by_cluster(df, cluster_num, vis_data: pyLDAvis.PreparedData):
    top_n = config.lda.top_n
    keywords = vis_data.topic_info["Term"].tolist()[:top_n]

    return keywords

In [ ]:
pyLDAvis.enable_notebook()
vis_data = gensimvis.prepare(lda_model, corpus, dictionary)
vis_path = f'./visualization/beauty.html'

pyLDAvis.display(vis_data)
pyLDAvis.save_html(vis_data, vis_path)

In [ ]:
pyLDAvis.display(vis_data)


In [ ]:
num_keywords = config.lda.num_keywords
keywords = vis_data.topic_info["Term"].tolist()[:num_keywords]
keywords

In [ ]:
df["keywords"] = [keywords for i in range(len(df))]
# set keywords column only in review_tokenized
df["a"] = df["review_tokenized"].apply(lambda x: set(x))
df["b"] = df["keywords"].apply(lambda x: set(x))
# intersection
df["keywords"] = df.apply(lambda x: list(x["a"] & x["b"]), axis=1)
df.drop(["a", "b"], axis=1, inplace=True)

In [ ]:
df = df[["review", "category", "keywords"]]

In [ ]:
df[["rawReview","review", "keywords"]].to_csv('./result_no_cluster.csv')

In [ ]:
from google.colab import files
files.download('./result_no_cluster.csv')

# Topic Modeling (Using C-TF-IDF;  Class-based TF-IDF)

C-TF-IDF를 통해서 각 묶어진 그룹(Topic 또는 Class)에 대해 해당 Topic을 잘 표현하는 단어를 찾는다.\
**버토픽 파이프라인 그대로 따라가는 中** .

In [ ]:
%%capture
!pip install bertopic

In [ ]:
# Data Load
# 앞서 구현한 SentenceTransformer 는 별도의 tuning 없는 과정이므로, BERTopic에 내장된 SentenceTransformer 그대로 사용
# distilbert-base-nli-stsb-mean-tokens

# 리뷰 원문 텍스트의 리스트화
docs = df["rawReview"].toarray()

### Class Preparation (삭제 예정)

In [ ]:
# Load data
docs = pd.DataFrame({'Document': df['review'], 'Class': df['clustered']})
docs_per_class = docs.groupby(['Class'], as_index=False).agg({'Document': ' '.join})

# Create bag of words
count_vectorizer = CountVectorizer().fit(docs_per_class.Document)
count = count_vectorizer.transform(docs_per_class.Document)
words = count_vectorizer.get_feature_names_out()

In [ ]:
import numpy as np
import scipy.sparse as sp

from sklearn.utils import check_array
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.utils.validation import FLOAT_DTYPES, check_is_fitted

# C-TF-IDF Vectorizer Class 를 정의
class CTFIDFVectorizer(TfidfTransformer):
    def __init__(self, *args, **kwargs):
        super(CTFIDFVectorizer, self).__init__(*args, **kwargs)
        self._idf_diag = None

    def fit(self, X: sp.csr_matrix, n_samples: int):
        """Learn the idf vector (global term weights)

        Parameters
        ----------
        X : sparse matrix of shape n_samples, n_features)
            A matrix of term/token counts.

        """

        # Prepare input
        X = check_array(X, accept_sparse=('csr', 'csc'))
        if not sp.issparse(X):
            X = sp.csr_matrix(X)
        dtype = X.dtype if X.dtype in FLOAT_DTYPES else np.float64

        # Calculate IDF scores
        _, n_features = X.shape
        df = np.squeeze(np.asarray(X.sum(axis=0)))
        avg_nr_samples = int(X.sum(axis=1).mean())
        idf = np.log(avg_nr_samples / df)
        self._idf_diag = sp.diags(idf, offsets=0,
                                  shape=(n_features, n_features),
                                  format='csr',
                                  dtype=dtype)
        return self

    def transform(self, X: sp.csr_matrix, copy=True) -> sp.csr_matrix:
        """Transform a count-based matrix to c-TF-IDF

        Parameters
        ----------
        X : sparse matrix of (n_samples, n_features)
            a matrix of term/token counts

        Returns
        -------
        vectors : sparse matrix of shape (n_samples, n_features)

        """

        # Prepare input
        X = check_array(X, accept_sparse='csr', dtype=FLOAT_DTYPES, copy=copy)
        if not sp.issparse(X):
            X = sp.csr_matrix(X, dtype=np.float64)

        n_samples, n_features = X.shape

        # idf_ being a property, the automatic attributes detection
        # does not work as usual and we need to specify the attribute
        # name:
        check_is_fitted(self, attributes=["idf_"],
                        msg='idf vector is not fitted')

        # Check if expected nr features is found
        expected_n_features = self._idf_diag.shape[0]
        if n_features != expected_n_features:
            raise ValueError("Input has n_features=%d while the model"
                             " has been trained with n_features=%d" % (
                                 n_features, expected_n_features))

        X = X * self._idf_diag

        if self.norm:
            X = normalize(X, axis=1, norm='l1', copy=False)

        return X


In [ ]:
ctfidf = CTFIDFVectorizer().fit_transform(count, n_samples=len(docs)).toarray()
ctfidf  # 확인 목적

In [ ]:
# Extract top 10 words # 10 은 변경 가능
ctfidf = CTFIDFVectorizer().fit_transform(count, n_samples=len(docs)).toarray()
words_per_class = {f'그룹 {label}': [words[index] for index in ctfidf[label].argsort()[-10:]] for label in docs_per_class.Class}

# 그룹 이름: [각 클러스터별 상위 10개 단어]
words_per_class

In [ ]:
docs # 확인 목적

,Document,Class
0,I was wondering if anyone out there could enli...,7
1,A fair number of brave souls who upgraded thei...,4
2,"well folks, my mac plus finally gave up the gh...",4
3,\nDo you have Weitek's address/phone number? ...,1
4,"From article <C5owCB.n3p@world.std.com>, by to...",14
...,...,...
11309,DN> From: nyeda@cnsvax.uwec.edu (David Nye)\nD...,13
11310,"I have a (very old) Mac 512k and a Mac Plus, b...",4
11311,I just installed a DX2-66 CPU in a clone mothe...,3
11312,\nWouldn't this require a hyper-sphere. In 3-...,1


In [ ]:
docs_per_class # 이때 클래스는 클러스터의 넘버를 의미한다.

,Class,Document
0,0,\nDon't be so sure. Look what happened to Jap...
1,1,\nDo you have Weitek's address/phone number? ...
2,2,I have win 3.0 and downloaded several icons an...
3,3,...
4,4,A fair number of brave souls who upgraded thei...
5,5,QUESTION:\n What is the EXACT entry (paramete...
6,6,Reduced Prices! \nI have a list of things fors...
7,7,I was wondering if anyone out there could enli...
8,8,I have a line on a Ducati 900GTS 1978 model wi...
9,9,Doug Roberts - Ken Hill for NL MVP!!\n\t ...


### Hyperparameter Tuning

**버토픽의 하이퍼파라미터 목록**

`top_n_words`: 각 토픽 별로 추출하고자 하는 단어의 개수 (확률분포값 기준, 상위 N개의 단어), 10과 20사이가 적절하고 값이 30을 넘기지 않는 것이 좋음. \
`n_gram_range`: 토픽 분포(representation)을 생성하는 CountVectorizer 에 반영되는 기준 단어 단위, 토픽별 추출되는 키워드를 구성하는 단어의 개수 \
`min_topic_size`: **(중요)** 하나의 토픽이 가져야 하는 최소 단어(키워드)의 개수, 이 값이 낮을수록 더 많은 토픽이 추출됨. (=제한조건이 낮으므로) / default 10  \
`nr_topics`: 토픽의 개수를 줄여서 결과적으로 남기고 싶은 토픽의 개수, 만약 "auto"로 설정한다면, 토픽의 개수를 HDBSCAN을 이용하여 자동적으로 특정 개수로 줄임. 너무 낮은 값으로 설정하면, 합쳐지지 말아야할 토픽들이 서로 합쳐져서 성능에 악영향을 미칠 수 있으니 주의. \


*`calculate_probabilities` 및 `low-memory` 하이퍼파라미터들은 일단 고려하지 않음.

각 하이퍼파라미터 세부설명 참고) \
 https://colab.research.google.com/drive/1ClTYut039t-LDtlcd-oQAdXWgcsSGTw9?usp=sharing#scrollTo=xLrIUdCGsgkf

In [ ]:
# Find best 'top_n_words' parameter by topic coherence score
coherence_score = []
num_topN_list = np.arange(5, 31, 5) # start, stop, step

for num_topN in num_topN_list:
    ctfidf_model = BERTopic(embedding_model=sentence_model, umap_model=umap_model, hdbscan_model=clusterer, top_n_words=num_topN).fit(docs) # 인자 이름 최종 확인 필요 (앞선 코드와의 매칭여부)
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                                id2word=dictionary,
                                                num_topics=num_topics,
                                                passes=passes,
                                                random_state=config.random_state)

    # Compute Coherence Score
    coherence_model_ctfidf = gensim.models.CoherenceModel(model=lda_model, texts=tokenized_text.tolist(), coherence='c_v')
    coherence_lda = coherence_model_ctfidf.get_coherence()
    print(f"num_topics={num_topics}, passes={num_passes}, Coherence Score={coherence_lda:.4f}")
    coherence_score.append(coherence_lda)
    del lda_model

### Extract Keywords

In [ ]:
# Customized BERTopic
# UMAP, HDBSCAN 의 경우 앞선, 튜닝 완료된 모델을 사용

ctfidf_model = BERTopic(embedding_model=sentence_model , umap_model=umap_model, hdbscan_model=hdbscan_model).fit(docs) # 인자 이름 최종 확인 필요 (앞선 코드와의 매칭여부)
ctfidf_model.get_topic_info().head(5)

In [ ]:
# Visualization
ctfidf_model.visualize_topics()

In [ ]:
ctfidf_model.visualize_distribution(probs[200], min_probability=0.015)

In [ ]:
ctfidf_model.visualize_barchart(top_n_topics=5)

**현재 각 (HDBSCAN 에 의해서 분류된) 각 클러스터별 상위 N 개의 키워드를 추출해낼 수 있는 상태까지 완료.** \
다만 본 테크닉의 경우 전체 문서 1개에 대해 키워드가 추출된 것이 아니라 '각 클러스터' 마다 추출된 것이기 때문에, \
이를 통해 각 리뷰별 키워드를 어떻게 추출해내는지 버토픽의 파이프라인을 더 공부해봐야할 것 같다.


*   추출하는 토픽의 개수 Coherence Score 비교 통해 찾기
*   추출하는 토픽 개수 조정할 수 있도록 코드 변경
*   각 리뷰별 해당하는 키워드 매칭시켜 하나의 df 로 완성

버토픽 상에서는, 최종적인 output 이 각 토픽별 리스트가 나와야한다.



